In [1]:
import util
import math
from functools import lru_cache
from tqdm import tqdm
import numpy as np
from collections import Counter

In [6]:
k, n = 20_000, 1_000_000

comb(n, n-k)

...

In [85]:
@lru_cache(maxsize=None)
def comb(n, r):
    if r >= n or n <= 1 or r <= 1: return math.comb(n, r)
    return math.comb(n, r) #n*comb(n, r)//(n-r)

@lru_cache(maxsize=None)
def perm(n, r):
    if r >= n or n <= 1 or r <= 1: return math.perm(n, r)
    #print(n, r, n-1, r-2)
    return n*(n-r+1)*perm(n-1, r-2)

@lru_cache(maxsize=None)
def p2(k, n):
    if n == 0: return 1
    return comb(k, 2) * p2(k-2, n-1)

@lru_cache(maxsize=None)
def p12(k, n):
    num = 0
    for twos in tqdm(range(k // 2 + 1)):
        kp = k-twos*2
        np = n-twos
        print(k, n, twos, kp, np, perm(np, kp), comb(n, np), comb(k, kp), p2(twos*2, twos))
        #num += perm(np, kp) * comb(n, np) * comb(k, kp) * p2(twos*2, twos)
        num += perm(np, kp) * comb(n, np) * comb(k, kp) * p2(twos*2, twos)
    return num

def p(k, n):
    return 1 - (p12(k, n) / n**k)

def p_sim(k, n):
    total = 200_000
    c = 0
    for _ in range(total):
        if any(v >= 3 for v in Counter(np.random.randint(n, size=k)).values()):
            c += 1
    return c / total

In [86]:
p(3, 7)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 1557.48it/s]

3 7 0 3 7 210 1 1 1
3 7 1 1 6 6 7 3 1


0.020408163265306145

In [27]:
%%time
k, n = 20_000, 1_000_000
twosr = range(k // 2 + 1)

for twos in tqdm(reversed(twosr)):
    perm(n-twos, k-twos*2)

for twos in twosr:
    p2(twos*2, twos)
    
p(k, n)

10001it [00:00, 62358.75it/s]
 12%|████████████▉                                                                                                | 1188/10001 [00:02<00:15, 562.75it/s]


KeyboardInterrupt: 

In [100]:
p_sim(17, 46)

0.239005

In [115]:
p(17, 46)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 609.98it/s]

46 17 45 15
45 15 44 13
44 13 43 11
43 11 42 9
42 9 41 7
41 7 40 5
40 5 39 3
39 3 38 1


0.23735911597746928

In [112]:
@lru_cache(maxsize=None)
def log_comb(n, r):
    if r >= n: return 0
    return log_perm(n, r) - sum(map(math.log, range(r, 0, -1)))

@lru_cache(maxsize=None)
def log_perm(n, r):
    if r >= n: return 0
    return sum(map(math.log, range(n, n - r, -1)))

@lru_cache(maxsize=None)
def log_p2(k, n):
    if k == 0 or n == 0: return 0
    return log_comb(k, 2) + log_p2(k-2, n-1)

In [137]:
log_combs = {}

def cache_log_comb(n, r):
    key = (n, r)
    prevkey = (n, r-1)
    if prevkey in log_combs:
        log_combs[key] = log_combs[prevkey] + math.log(n-r+1) - math.log(r)
    else:
        log_combs[key] = log_comb(n, r)

def fill_caches(k, n):
    twosr = range(k // 2 + 1)
    for twos in reversed(twosr):
        cache_log_comb(n, n-twos)
        cache_log_comb(k, k-twos*2)
        log_perm(n-twos, k-twos*2)

    for twos in twosr:
        log_p2(twos*2, twos)

In [138]:
def log_p12(k, n):
    fill_caches(k, n)
    nums = []
    log_d = math.log(n)*k
    for twos in tqdm(range(k // 2 + 1)):
        kp = k-twos*2
        np = n-twos
        if np < kp: continue
        a, b, c, d = log_perm(np, kp), log_combs[(n, np)], log_combs[(k, kp)], log_p2(twos*2, twos)
        nums.append(a + b + c + d - log_d)
    return sum(map(math.exp, nums))

def p(k, n):
    return 1 - log_p12(k, n)

In [139]:
p(3, 7)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 25497.29it/s]


0.020408163265305923

In [140]:
%%time
k, n = 20_000, 1_000_000
p(k, n)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 10001/10001 [00:00<00:00, 679208.44it/s]

CPU times: user 9.17 s, sys: 31.9 ms, total: 9.21 s
Wall time: 9.21 s


0.7311720309416538

In [4]:
combs = {}

def cache_comb(n, r):
    key = (n, r)
    prevkey = (n, r-1)
    pprevkey = (n, r-2)
    if prevkey in combs:
        combs[key] = (combs[prevkey] * (n-r+1)) // r
    else:
        combs[key] = math.comb(n, r)

@lru_cache(maxsize=None)
def perm(n, r):
    if r >= n or n <= 1 or r <= 1: return math.perm(n, r)
    return n*(n-r+1)*perm(n-1, r-2)

@lru_cache(maxsize=None)
def comb(n, r):
    if r >= n or n <= 1 or r <= 1: return math.comb(n, r)
    return combs[(n, r)]

@lru_cache(maxsize=None)
def p2(k, n):
    if n == 0: return 1
    return math.comb(k, 2) * p2(k-2, n-1)

def fill_caches(k, n):
    twosr = range(k // 2 + 1)
    for twos in reversed(twosr):
        cache_comb(n, n-twos)
        if twos * 2 != k:
            cache_comb(k, k-twos*2-1)
        cache_comb(k, k-twos*2)
        perm(n-twos, k-twos*2)

    for twos in twosr:
        p2(twos*2, twos)

def p12(k, n):
    print('caching...')
    fill_caches(k, n)
    print('done caching!')
    nums = []
    log_d = math.log(n)*k
    for twos in tqdm(range(k // 2 + 1)):
        kp = k-twos*2
        np = n-twos
        if np < kp: continue
        parts = perm(np, kp), comb(n, np), comb(k, kp), p2(twos*2, twos)
        nums.append(sum(map(math.log, parts)) - log_d) 
    return sum(map(math.exp, nums))

def p(k, n):
    return 1 - p12(k, n)

In [6]:
%%time
k, n = 20_000, 1_000_000

p(k, n)

caching...
done caching!


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 10001/10001 [00:00<00:00, 297320.99it/s]

CPU times: user 592 ms, sys: 16.3 ms, total: 609 ms
Wall time: 605 ms


0.7311720251209547